In [4]:
import pandas as pd
df = pd.read_csv("./data/meteo/2013/01.csv.gz", compression="gzip", header=None, sep=";")
print(df.head())

          0               1       2     3         4    5         6   \
0  numer_sta            date    pmer  tend  cod_tend   dd        ff   
1      07005  20130101000000  100290   -50         8  200  6.700000   
2      07015  20130101000000  100270   -80         6  210  8.700000   
3      07020  20130101000000  100350    40         3  260  7.700000   
4      07027  20130101000000  100370   -60         6  220  4.600000   

           7           8   9   ...       50      51       52       53      54  \
0           t          td   u  ...  nnuage2  ctype2  hnuage2  nnuage3  ctype3   
1  281.650000  280.450000  92  ...       mq      mq       mq       mq      mq   
2  281.450000  280.050000  91  ...       mq      mq       mq       mq      mq   
3  284.150000  281.850000  86  ...       mq      mq       mq       mq      mq   
4  282.350000  280.950000  91  ...       mq      mq       mq       mq      mq   

        55       56      57       58  59  
0  hnuage3  nnuage4  ctype4  hnuage4 NaN  


In [ ]:
68 042 591
67 842 591
67 635 124
67 441 85
67 257 982
66 992 159
66 774 482
66 602 645
66 422 469
66 130 873
65 564 756
65 241 241
64 933 4
64 612 939
64 304 5
63 961 859
63 600 69
63 186 117
62 730 537
62 251 062
61 824 03
61 385 07
60 941 41
60 508 15
60 122 665
59 899 347
59 691 177
59 487 413
59 280 577
59 070 077
58 852 002
58 571 237
58 280 135
57 996 401
